In [ ]:
import requests
import zipfile
import pathlib
import os
import shutil

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tarfile

try:
    from PIL import Image
except:
    print (f'[INFO] Couldn\'t find pillow installed...... Installing it')
    !pip3 install -q pillow
    from PIL import Image

#importing machine learning moudules and libraries

try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1])>=12
    assert int(torch.__version__.split(".")[1])>=13
    print(f'torch version: {torch.__version__}')
    print(f" torchvision version: {torchvision.__version__}")
except:
  print(f"torch version not up to date....... Installing updated version \n")
  print(f"torchvision version not up to date........ Installing the updated version")
  !pip3 install -U torch, torchvision, torchaudion --extra-index-url https://download.pytorch.org/whl/cu112
  import torch
  import torchvision
  print(f"torch version: {torch.__version__}")
  print(f"torchvision version: {torchvision.__version__}")

from torchvision import datasets, transforms
from torch.utils import tensorboard
from torch.utils.data import DataLoader
from torch import nn

try:
    from torchinfo import summary
except:
    print(f'[INFO] torchinfo not installed....installing')
    !pip install -q torchinfo
    from torchinfo import summary


torch version not up to date....... Installing updated version 

torchvision version not up to date........ Installing the updated version
ERROR: Invalid requirement: 'torch,'
torch version: 2.1.0+cu118
torchvision version: 0.16.0+cu118
[INFO] torchinfo not installed....installing


In [ ]:
# Get training data
data_download = pathlib.Path("/content/data")
train_data = datasets.Food101(root=data_download,
                              split="train",
                              # transform=transforms.ToTensor(),
                              download=True)

# Get testing data
test_data = datasets.Food101(root=data_download,
                             split="test",
                             # transform=transforms.ToTensor(),
                             download=True)

100%|██████████| 4996278331/4996278331 [11:00<00:00, 7570056.32it/s]


Extracting /content/data/food-101.tar.gz to /content/data


In [ ]:
# Get random 10% of training images
import random

# Setup data paths
data_path1 = data_download / "food-101" / "images"
target_classes = ["pizza", "steak", "sushi", "tacos"]

# Change amount of data to get (e.g. 0.1 = random 10%, 0.2 = random 20%)
amount_to_get = 0.2

# Create function to separate a random amount of data
def get_subset(image_path=data_path1,
               data_splits=["train", "test"],
               target_classes=["pizza", "steak", "sushi","tacos"],
               amount=0.1,
               seed=42):
    random.seed(42)
    label_splits = {}

    # Get labels
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        label_path = data_download / "food-101" / "meta" / f"{data_split}.txt"
        with open(label_path, "r") as f:
            labels = [line.strip("\n") for line in f.readlines() if line.split("/")[0] in target_classes]

        # Get random subset of target classes image ID's
        number_to_sample = round(amount * len(labels))
        print(f"[INFO] Getting random subset of {number_to_sample} images for {data_split}...")
        sampled_images = random.sample(labels, k=number_to_sample)

        # Apply full paths
        image_paths = [pathlib.Path(str(image_path / sample_image) + ".jpg") for sample_image in sampled_images]
        label_splits[data_split] = image_paths
    return label_splits

label_splits = get_subset(amount=amount_to_get)

[INFO] Creating image split for: train...
[INFO] Getting random subset of 600 images for train...
[INFO] Creating image split for: test...
[INFO] Getting random subset of 200 images for test...


In [ ]:
# Create target directory path
target_dir_name = f"/content/data/pizza_steak_sushi_tacos_{str(int(amount_to_get*100))}_percent"
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories
target_dir = pathlib.Path(target_dir_name)

# Make the directories
target_dir.mkdir(parents=True, exist_ok=True)

Creating directory: '/content/data/pizza_steak_sushi_tacos_20_percent'


In [ ]:
for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parent.stem / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        print(f"[INFO] Copying {image_path} to {dest_dir}...")
        shutil.copy2(image_path, dest_dir)

[INFO] Copying /content/data/food-101/images/tacos/2746697.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/tacos/2746697.jpg...
[INFO] Copying /content/data/food-101/images/pizza/3269634.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/pizza/3269634.jpg...
[INFO] Copying /content/data/food-101/images/pizza/1524655.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/pizza/1524655.jpg...
[INFO] Copying /content/data/food-101/images/steak/2825100.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/steak/2825100.jpg...
[INFO] Copying /content/data/food-101/images/steak/225990.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/steak/225990.jpg...
[INFO] Copying /content/data/food-101/images/steak/1839481.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/steak/1839481.jpg...
[INFO] Copying /content/data/food-101/images/pizza/38349.jpg to /content/data/pizza_steak_sushi_tacos_20_percent/train/pizza/38349.jpg...
[INFO] Copyi

In [ ]:
# Setup train and testing paths
train_dir = target_dir / "train" #path to only the training folder
test_dir = target_dir / "test"   #path to only the test folder

train_dir, test_dir

(PosixPath('/content/data/pizza_steak_sushi_tacos_20_percent/train'),
 PosixPath('/content/data/pizza_steak_sushi_tacos_20_percent/test'))

In [ ]:
data_path = data_download  #path to all the data
image_path = target_dir    #path to folder containing train and test folder.
image_path

PosixPath('/content/data/pizza_steak_sushi_tacos_20_percent')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#creating ViT model

def create_vit_model(num_classes: int = 4,
                     seed: int = 42):
  ''' Creates a ViT-B/16 feature extractor model and transforms.

  Args:
      num_classes (int, optional): number of target classes. Defaults to 4.
      seed (int, optional): random seed value for output layer. Defaults to 42.

  Returns:
          model (torch.nn.Module): ViT-B/16 feature extractor model.
          transforms (torchvision.transforms): ViT-B/16 image transforms.
  '''
  #create ViT-B/16 pretrained weights, transforms and model
  weights = torchvision.models.ViT_B_16_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.vit_b_16(weights = weights)

  #freezing all layers of the model
  for param in model.parameters():
      param.requires_grad = False
  torch.manual_seed(seed)
  model.heads = nn.Sequential(nn.Linear(in_features = 768,
                                        out_features = num_classes))

  return model, transforms

In [ ]:
#creating the ViT model
vit, vit_transforms = create_vit_model()

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:05<00:00, 69.0MB/s]


In [ ]:
summary(vit,
        input_size=(1, 3, 224, 224),
        col_names = ["input_size", "output_size", "num_params", "trainable"],
        col_width = 20,
        row_settings = ["var_names"]
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [1, 3, 224, 224]     [1, 4]               768                  Partial
├─Conv2d (conv_proj)                                         [1, 3, 224, 224]     [1, 768, 14, 14]     (590,592)            False
├─Encoder (encoder)                                          [1, 197, 768]        [1, 197, 768]        151,296              False
│    └─Dropout (dropout)                                     [1, 197, 768]        [1, 197, 768]        --                   --
│    └─Sequential (layers)                                   [1, 197, 768]        [1, 197, 768]        --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [1, 197, 768]        [1, 197, 768]        (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [1, 197, 768]        [1, 1

In [ ]:
#creating a dataloader object
#notice that the return annotation is extracted directly from the docstring.Tuple[DataLoader, DataLoader, List[str]]:
NUM_WORKERS = os.cpu_count()
def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS):
  ''' Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into a Pytorch Dataset Object, then to pytorch Dataloaders

  Args:
      train_dir: Path object to training directory.
      test_dir:  Path object to test directory.
      transform: transformation to perform on data objects
      num_workers: An integer for number of workers per DataLoader.

  Returns:
      A tuple of (train_dataloader, test_dataloader, class_names).
      Where class_names is a list of the target classes.
      Example uasge:
        train_dataloader, test_dataloader, class_names =\
        = create_dataloaders(train_dir = path/ to/train_dir,
                              test_dir = path/to/test_dir,
                              transform = torchvision.transfroms,
                              batch_size = 32,
                              num_workers = 4 )

  '''
  #Using ImageFolder() to create a dataset object
  train_data = datasets.ImageFolder(train_dir, transform = transform)
  test_data = datasets.ImageFolder(test_dir, transform = transform)

  #Get class names
  class_names = train_data.classes #remember that ImageFolder maps class-to-idx

  #Turn images into dataloaders
  train_dataloader = DataLoader(train_data,
                                batch_size = batch_size,
                                shuffle = True,
                                num_workers =num_workers,
                                pin_memory = True)
  test_dataloader = DataLoader(test_data,
                               batch_size = batch_size,
                               shuffle = False,
                               num_workers = num_workers,
                               pin_memory = True)

  return train_dataloader, test_dataloader, class_names



In [ ]:
# creating dataloaders
train_dataloader_vit, test_dataloader_vit, class_names = create_dataloaders(train_dir,
                                                                            test_dir,
                                                                            vit_transforms,
                                                                            batch_size = 32)

In [ ]:
# define the train and test functions
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [ ]:
#define a seed function

def set_seeds(seed: int=42):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [ ]:
#train our model

#setup the optimizer
optimizer = torch.optim.Adam(params = vit.parameters(),
                             lr = 1e-3)

#setup the loss function
loss_fn = torch.nn.CrossEntropyLoss()

set_seeds()
vit_results = train(model = vit,
                    train_dataloader = train_dataloader_vit,
                    test_dataloader = test_dataloader_vit,
                    epochs = 10,
                    optimizer = optimizer,
                    loss_fn = loss_fn,
                    device = device)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.7575 | train_acc: 0.7845 | test_loss: 0.3838 | test_acc: 0.9107
Epoch: 2 | train_loss: 0.2778 | train_acc: 0.9260 | test_loss: 0.2705 | test_acc: 0.9241
Epoch: 3 | train_loss: 0.2019 | train_acc: 0.9375 | test_loss: 0.2402 | test_acc: 0.9196
Epoch: 4 | train_loss: 0.1662 | train_acc: 0.9430 | test_loss: 0.2190 | test_acc: 0.9196
Epoch: 5 | train_loss: 0.1393 | train_acc: 0.9600 | test_loss: 0.2089 | test_acc: 0.9152
Epoch: 6 | train_loss: 0.1194 | train_acc: 0.9720 | test_loss: 0.2026 | test_acc: 0.9241
Epoch: 7 | train_loss: 0.1048 | train_acc: 0.9764 | test_loss: 0.1961 | test_acc: 0.9241
Epoch: 8 | train_loss: 0.0904 | train_acc: 0.9830 | test_loss: 0.1902 | test_acc: 0.9286
Epoch: 9 | train_loss: 0.0793 | train_acc: 0.9868 | test_loss: 0.1878 | test_acc: 0.9330
Epoch: 10 | train_loss: 0.0707 | train_acc: 0.9934 | test_loss: 0.1846 | test_acc: 0.9330


In [ ]:
# define a function to save the model

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  #create a target directory
  target_dir_path = pathlib.Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok = True)

  #create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  #saving the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj = model.state_dict(),
             f=model_save_path)

In [ ]:
save_model(model = vit,
           target_dir = 'models',
           model_name = "pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth")

[INFO] Saving model to: models/pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth


In [ ]:
vit.to("cpu")

#check the device
next(iter(vit.parameters())).device

device(type='cpu')

In [ ]:
from timeit import default_timer as timer

In [ ]:
#create a gradio predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns pretiction and time taken.
  """
  #start the timer
  start_time = timer()

  # Transform the target image and add a batch dimension
  img = vit_transforms(img).unsqueeze(0)

  #put model into evaluation mode and turn on inference mode
  vit.eval()

  with torch.inference_mode():
    #pass the transformed image through the model and turn the prediction logits into predicted probabilitieis
    pred_probs = torch.softmax(vit(img), dim = 1)

  #create a predicition label and prediction probability dictionary for each prediction class
  #this is the required format for gradio's output parameters
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}

  #calculate the prediction time
  pred_time = round(timer()- start_time, 5)

  #Return the predicted dictionary and predicted time
  return pred_labels_and_probs, pred_time


In [ ]:
import random
#create a list of examples as required by gradio
test_data_paths = list(pathlib.Path(test_dir).glob("*/*.jpg"))
examples_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
examples_list

[['/content/data/pizza_steak_sushi_tacos_20_percent/test/steak/2510026.jpg'],
 ['/content/data/pizza_steak_sushi_tacos_20_percent/test/steak/3873283.jpg'],
 ['/content/data/pizza_steak_sushi_tacos_20_percent/test/sushi/3196729.jpg']]

In [ ]:
#Build the Gradio Interface

import gradio as gr
#create title, description and article strings
title = 'FoodClassifier Mini'
description = "A Vision Transformer feature extractor computer vision model to classify images of food as pizza, steak, sushi or tacos."
article = "Created by [Emerald Henry](https://henrii1.github.io)."

# Create the Gradio demo
demo = gr.Interface(fn=predict,
                    inputs = gr.Image(type="pil"),
                    outputs = [gr.Label(num_top_classes=4, label="Predictions"),
                               gr.Number(label = "Prediction time (s)")],
                    examples = examples_list,
                    title = title,
                    description = description,
                    article = article)

# Launch the demo!

demo.launch(debug=False,
            share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f4bd1803cc39607f73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
#!pip install -q gradio

In [ ]:
#import gradio as gr

In [ ]:
#creating the app in hugging space
#creating a demos folder to store out food vision mini app files

import shutil
from pathlib import Path

#create foodvision mini demo path
foodvision_mini_demo_path = Path("demos/foodvision_mini/")

#Remove files that might already exist there and create new directory

if foodvision_mini_demo_path.exists():
  shutil.rmtree(foodvision_mini_demo_path)
  foodvision_mini_demo_path.mkdir(parents = True,
                                  exist_ok = True)
else:
  foodvision_mini_demo_path.mkdir(parents = True,
                                  exist_ok = True)

!ls demos/foodvision_mini/

In [ ]:
#creating a folder of examples

#create an example directory
foodvision_mini_examples_path = foodvision_mini_demo_path / "examples"
foodvision_mini_examples_path.mkdir(parents = True,
                                    exist_ok = True)

#collect three random dataset image paths
foodvision_mini_examples = [Path('data/pizza_steak_sushi_tacos_20_percent/test/sushi/1016390.jpg'),
                            Path('data/pizza_steak_sushi_tacos_20_percent/test/steak/1064847.jpg'),
                            Path('data/pizza_steak_sushi_tacos_20_percent/test/pizza/1106738.jpg')]

#copy the three random images to the examples directory
for example in foodvision_mini_examples:
  destination = foodvision_mini_examples_path / example.name
  shutil.copy2(src = example, dst = destination)

In [ ]:
#moving the model to our foodvision directory
vit_foodvision_mini_model_path = 'models/pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth'

#create a destination path for our target model
vit_foodvision_mini_model_destination = foodvision_mini_demo_path /vit_foodvision_mini_model_path.split("/")[1]


#Try to move the file

try:
  print(f"[INFO] Attempting to move {vit_foodvision_mini_model_path} to {vit_foodvision_mini_model_destination}")
  #move the file
  shutil.move(src=vit_foodvision_mini_model_path,
              dst = vit_foodvision_mini_model_destination)
  print(f"[INFO] Model move complete.")

except:
    print(f"[INFO] No model found at {vit_foodvision_mini_model_path}, perhaps its already been moved?")
    print(f"[INFO] Model exists at {vit_foodvision_mini_model_destination}: {vit_foodvision_mini_model_destination.exists()}")



[INFO] Attempting to move models/pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth to demos/foodvision_mini/pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth
[INFO] Model move complete.


In [ ]:
%%writefile demos/foodvision_mini/model.py
import torch
import torchvision

from torch import nn

def create_vit_model1(num_classes: int = 4,
                      seed: int =42)
  """Creates an ViT feature extractor model and transforms.

  Args:
        num_classes (int, optional): number of classes in the classifier head.
            Defaults to 4.
        seed (int, optional): random seed value. Defaults to 42.

  Returns:
        model (torch.nn.Module): ViT feature extractor model.
        transforms (torchvision.transforms): ViT image transforms.
  """
  #create vit pretrained weights and transforms and model
  weights =torchvision.models.ViT_B_16_Weights.DEFAULT
  transforms = weights.transforms()
  model = torchvision.models.vit_b_16(weights = weights)

  # Freeze all layers in base model
  for param in model.parameters():
      param.requires_grad = False

  #Change classifier head with random seed for reproducibility
  torch.manual_seed(seed)
  model.classifier = nn.Sequential(nn.Dropout(p=0.3, inplace = True),
                                    nn.Linear(in_features = 1408, out_features = num_classes))
  return model, transforms

Writing demos/foodvision_mini/model.py


In [ ]:
#turn gradio into written file
%%writefile demos/foodvision_mini/app.py
#import moudules
import gradio as gr
import os
import torch

from model import create_vit_model1
from timeit import default_timer as timer
from typing import Tuple, Dict

#setup class names
class_names = ["pizza", "steak", "sushi", "Tacos"]

#model and transform preparation

#create model
vit, vit_transforms1 = create_vit_model1(num_classes = 4)

#load saved weights
vit.load_state_dict(torch.load(f= 'models/pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth',
                                map_location = torch.device("cpu") )

)

#predict function
def predict(img) -> Tuple[Dict, float]:
  """Transforms and performs a prediction on img and returns prediction and time taken.
  """
  # start the timer
  start_time = timer()

  #Transform the target image and add a batch dimension
  img = vit_transforms1(img).unsqueeze(0)

  #put model into evaluation mode and turn on inference model
  vit.eval()
  with torch.inference_mode():
    #pass the transformed image throug the model and turn prediction to probabilites
    pred_probs = torch.softmax(vit(img), dim = 1)

  #create a prediction label and prediction probability dic for each class, as required by gradio
  pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i] ) for i in range(len(class_names))}

  #Calculate the prediction time
  pred_time = round(timer() - start_time, 5)

  return pred_labels_and_probs, pred_time



#Gradio App

#create title, description and article strings

title = "FoodClassifier Mini"
description = "A ViT feature extractor computer vision model to classify images of food as pizza, steak, sushi or Tacos"
article = created by [Emerald Henry](https://henrii1.github.io)

example_list = [["examples/" + example] for example in os.listdir("examples")]

#create the gradio demo

demo = gr. Interface(fn= predict,
                      inputs = gr.Image(type = "pil,
                      outpus = [gr.Label(num_top_classes = 4, label='Preciction,
                                gr.Number(label = "Prediction time (s,
                      examples = example_list,
                      title = title,
                      description = description,
                      article = article)
demo.launch()


Writing demos/foodvision_mini/app.py


In [ ]:
%%writefile demos/foodvision_mini/requirement.txt

torch==1.12.0
torchvision == 0.13.0
gradio == 3.1.4

Writing demos/foodvision_mini/requirement.txt


In [ ]:
zip -r ../foodvision_mini.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"


SyntaxError: ignored

In [ ]:
#download the file were working with

!cd demos/foodvision_mini && zip -r ../foodvision_mini.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

# Download the zipped Food vision mini app.

try:
  from google.colab import files
  files.download("demos/foodvision_mini.zip")
except:
  print("Not running in Google Colab, can't use google.colab.files.download(), please download manually")

  adding: app.py (deflated 55%)
  adding: examples/ (stored 0%)
  adding: examples/1106738.jpg (deflated 0%)
  adding: examples/1064847.jpg (deflated 0%)
  adding: examples/1016390.jpg (deflated 1%)
  adding: model.py (deflated 54%)
  adding: pretrained_vit_feature_extractor_pizza_steak_sushi_tacos_20_percent.pth (deflated 7%)
  adding: requirement.txt (deflated 9%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>